Pobieranie danych historycznych NOAA - dane treningowe

Wczytanie i oczyszczenie danych

In [ ]:
import pandas as pd
import numpy as np


# Wczytanie i oczyszczenie danych

data = pd.read_csv("Dane_NOAA.csv")

weather_data = data[["DATE", "TEMP", "PRCP", "WDSP", "MAX", "MIN"]].dropna()

#np.random.seed(42)
#anomaly_indices = np.random.choice(len(weather_data), size=10, replace=False)                Ewentualnie dodane sztucznie anomalie
#weather_data.loc[anomaly_indices, "TEMP"] *= 1.5  # sztuczne przegrzanie
#weather_data.loc[anomaly_indices, "PRCP"] *= 3    # sztuczne ulewy

weather_data.to_csv("obrobione_Dane_NOAA.csv", index=False)


Model 1 - Klasyfikacja ekstremów (Drzewo Decyzyjne)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Definicja ekstremów (0 = normalny dzień, 1 = upał, 2 = ulewa, 3 = wichura)

weather_data["EXTREME"] = 0  #brak ekstremum
weather_data.loc[weather_data["MAX"] > 32, "EXTREME"] = 1          # upał gdy >32°C
weather_data.loc[weather_data["PRCP"] > 20, "EXTREME"] = 2         # ulewa gdy >20mm
weather_data.loc[weather_data["WDSP"] > 30, "EXTREME"] = 3         # wichura gdy >30 węzłów

X = weather_data[["TEMP", "PRCP", "WDSP"]]
y = weather_data["EXTREME"]

# Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Trenowanie modelu
model_dt = DecisionTreeClassifier(max_depth=4)
model_dt.fit(X_train, y_train)

# Ocena
print("Accuracy:", model_dt.score(X_test, y_test))


Autoencoder do wykrywania nietypowych wzorców

Przygotowanie danych

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Wczytanie danych
data = pd.read_csv("Dane_NOAA.csv")
features = data[["TEMP", "PRCP", "WDSP"]]

# Normalizacja do przedziału [0,1]
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(features)

Model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import MinMaxScaler

# Normalizacja danych
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(weather_data[["TEMP", "PRCP", "WDSP"]])

# Budowa autoencodera
input_layer = Input(shape=(3,))
encoded = Dense(2, activation='relu')(input_layer)  # warstwa ukryta
decoded = Dense(3, activation='sigmoid')(encoded)  # rekonstrukcja

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Trenowanie
autoencoder.fit(X_scaled, X_scaled, epochs=50, batch_size=32)

# Obliczanie błędów rekonstrukcji
reconstructions = autoencoder.predict(X_scaled)
mse = np.mean(np.power(X_scaled - reconstructions, 2), axis=1)
weather_data["AE_ANOMALY"] = mse > np.quantile(mse, 0.95)  # górne 5% to anomalie

Kompilacja i trening

In [ ]:
autoencoder.compile(optimizer="adam", loss="mse")  # MSE - błąd średniokwadratowy
history = autoencoder.fit(
    X_normalized, X_normalized,  # Autoencoder uczy się rekonstruować własne wejście
    epochs=100,
    batch_size=32,
    validation_split=0.2
)

Wykrywanie anomalii

In [ ]:
# Rekonstrukcja danych
reconstructions = autoencoder.predict(X_normalized)

# Obliczenie błędu rekonstrukcji dla każdej próbki
mse = np.mean(np.power(X_normalized - reconstructions, 2), axis=1)

# Określenie progu anomalii (np. 95 percentyl)
threshold = np.quantile(mse, 0.95)

# Znajdowanie anomalii
anomalies = mse > threshold

Wizualizacja danych

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(mse, "o", markersize=4, label="Normalne dni")
plt.plot(np.where(anomalies)[0], mse[anomalies], "ro", 
         markersize=5, label="Anomalie")
plt.axhline(threshold, color="r", linestyle="--", label="Próg anomalii")
plt.xlabel("Numer dnia")
plt.ylabel("Błąd rekonstrukcji (MSE)")
plt.legend()
plt.show()

Wykrywanie anomalii (Isolation Forest)

In [ ]:
from sklearn.ensemble import IsolationForest

# Dane wejściowe (tylko temperatura i opady)
X = weather_data[["TEMP", "PRCP"]]

# Model Isolation Forest
model_if = IsolationForest(contamination=0.05, random_state=42)  # zakładamy 5% anomalii
model_if.fit(X)

# Predykcja (-1 = anomalia, 1 = normalny dzień)
anomalies = model_if.predict(X)
weather_data["ANOMALY"] = anomalies

# Filtrowanie anomal
anomaly_days = weather_data[weather_data["ANOMALY"] == -1]
print("Wykryte anomalie:", anomaly_days[["DATE", "TEMP", "PRCP"]])